In [ ]:
# -*- coding: utf-8 -*-
import os
import random
import numpy as np
import tensorflow as tf
from PIL import Image, ImageEnhance
from matplotlib import pyplot as plt

# TensorFlow and Keras
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, Callback
from tensorflow.keras.metrics import (
    AUC, Precision, Recall, 
    TruePositives, TrueNegatives, 
    FalsePositives, FalseNegatives
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3

# Albumentations for advanced image augmentation
from albumentations import (
    HueSaturationValue, RGBShift, RandomRotate90, 
    RandomFog, RandomRain, RandomSnow, Spatter, 
    GaussNoise, CLAHE, CoarseDropout, MedianBlur, PixelDropout
)

In [ ]:
# Define Albumentations augmentations
rotate_aug = RandomRotate90(always_apply=True, p=1.0)
hue_sat_val = HueSaturationValue(p=1.0, sat_shift_limit=(-5, 20), val_shift_limit=(-5, 10))
gauss_noise = GaussNoise(p=1.0, var_limit=(10.0, 100.0), per_channel=True)
clahe = CLAHE(p=1.0, clip_limit=(1, 5), tile_grid_size=(10, 10))
coarse_dropout = CoarseDropout(p=1.0, max_holes=30, max_height=10, max_width=10, min_holes=10, min_height=8, min_width=8)
pixel_dropout = PixelDropout(p=1.0, dropout_prob=0.05, per_channel=True)
splatter_configs = [
    Spatter(p=1.0, mean=(0.46, 0.46), std=(2.3, 2.3), gauss_sigma=(0.99, 0.99), intensity=(0.2, 0.2), mode=["rain"]),
    Spatter(p=1.0, mean=(0.66, 0.66), std=(0.3, 0.3), gauss_sigma=(0.33, 0.33), intensity=(0.39, 0.39), mode=["rain"]),
    Spatter(p=1.0, mean=(0.65, 0.65), std=(0.3, 0.3), gauss_sigma=(3.16, 3.16), intensity=(0.33, 0.33), mode=["rain"])
]
fog_configs = [
    RandomFog(p=1.0, fog_coef_lower=0.1, fog_coef_upper=0.4, alpha_coef=0.5),
    RandomFog(p=1.0, fog_coef_lower=0.04, fog_coef_upper=0.32, alpha_coef=0.94)
]
random_snow = RandomSnow(p=1.0, snow_point_lower=0.1, snow_point_upper=0.25, brightness_coeff=1.2)
random_rain_configs = [
    RandomRain(p=1.0, slant_lower=-5, slant_upper=5, drop_length=20, drop_width=1, rain_type="drizzle"),
    RandomRain(p=1.0, slant_lower=0, slant_upper=0, drop_length=5, drop_width=5, drop_color=(200, 200, 220), blur_value=7, brightness_coefficient=0.77)
]

# Distortion augmentation function
def distortion_augment(image):
    augmentations = [
        lambda x: fog_configs[0](image=x)["image"],
        lambda x: random_rain_configs[0](image=x)["image"],
        lambda x: random_snow(image=x)["image"],
        lambda x: splatter_configs[0](image=x)["image"],
        lambda x: gauss_noise(image=x)["image"],
        lambda x: clahe(image=x)["image"],
        lambda x: coarse_dropout(image=x)["image"],
        lambda x: pixel_dropout(image=x)["image"],
        lambda x: splatter_configs[1](image=x)["image"],
        lambda x: fog_configs[1](image=x)["image"],
        lambda x: splatter_configs[2](image=x)["image"],
        lambda x: random_rain_configs[1](image=x)["image"]
    ]
    return random.choice(augmentations)(image)

# Color augmentation function
def color_augment(image):
    if random.choice([True, False]):
        # RGB enhancement
        pil_image = Image.fromarray(np.uint8(image))
        enhancer = ImageEnhance.Color(pil_image)
        enhanced_image = enhancer.enhance(2)
        return np.asarray(enhanced_image, dtype=np.uint8)
    else:
        return hue_sat_val(image=image)["image"]

# Preprocess with augmentation
def preprocess_input_augment(image):
    """Apply augmentations and normalize the image."""
    image = np.uint8(image)
    
    # Always apply rotation
    image = rotate_aug(image=image)["image"]

    # Apply random augmentations
    if random.random() < 0.4:
        pass  # Do nothing
    elif random.random() < 0.6:
        image = color_augment(image)
    else:
        image = distortion_augment(image)

    # Normalize the image
    image = (image / 255.0 - 0.5) * 2.0
    return image

# Preprocess without augmentation
def preprocess_input(image):
    """Normalize the image without augmentation."""
    return (image / 255.0 - 0.5) * 2.0

In [ ]:
def save_model(model, prefix):
    """
    Save the model's weights and architecture to files.

    Args:
        model (keras.Model): The model to save.
        prefix (str): The file prefix for saving the model.
    """
    # Save the model's weights
    weights_path = f"{prefix}.h5"
    model.save_weights(weights_path)
    print(f"Model weights saved to: {weights_path}")

    # Save the model's architecture in JSON format
    architecture_path = f"{prefix}.json"
    with open(architecture_path, "w") as json_file:
        json_file.write(model.to_json())
    print(f"Model architecture saved to: {architecture_path}")


def count_files_in_folder(folder_path):
    """
    Count the total number of files in a folder and its subfolders.

    Args:
        folder_path (str): The path to the folder.

    Returns:
        int: The total number of files.
    """
    return sum(len(files) for _, _, files in os.walk(folder_path))


In [ ]:
# Configuration parameters
batch_size = 16  # Number of images per batch (low due to high image size: 384x384px)
nr_epochs = 1000
img_width, img_height = 384, 384
input_shape = (img_width, img_height, 3)

# Model and dataset details
model_name = "inception_v3"
train_data_dir = "/trainset/"
validation_data_dir = "/valset"
nr_classes = 2
tags = ["positive", "negative"]

# Count the number of training and validation samples
nr_train_samples = count_files_in_folder(train_data_dir)
nr_val_samples = count_files_in_folder(validation_data_dir)

# Dataset information
print("Loading dataset:")
print(f"Number of training samples: {nr_train_samples}")
print(f"Number of validation samples: {nr_val_samples}")

# Set image data format
K.set_image_data_format("channels_last")

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input_augment,
    brightness_range=[0.97, 1.03],
    horizontal_flip=True,
    vertical_flip=True
)

# Data preprocessing for validation
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

# Training data generator
train_generator = train_datagen.flow_from_directory(
    directory=train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

# Validation data generator
validation_generator = validation_datagen.flow_from_directory(
    directory=validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True
)

In [ ]:
def build_inception_model(input_shape, num_classes=2):
    """
    Builds and compiles an InceptionV3 model with a custom top layer.

    Args:
        input_shape (tuple): The shape of the input images (height, width, channels).
        num_classes (int): Number of output classes (default: 2).

    Returns:
        keras.Model: The compiled model.
    """
    # Initialize the base InceptionV3 model
    base_model = InceptionV3(weights=None, input_shape=input_shape, include_top=False)
    base_model.trainable = True

    # Add the custom top layers
    head_model = base_model.output
    head_model = GlobalAveragePooling2D()(head_model)
    head_model = Dropout(0.3)(head_model)
    head_model = Dense(64, activation="relu", kernel_regularizer=regularizers.l2(0.01))(head_model)
    head_model = Dropout(0.3)(head_model)
    output_layer = Dense(num_classes - 1, activation="sigmoid")(head_model)

    # Build the model
    model = Model(inputs=base_model.input, outputs=output_layer)

    # Ensure all layers are trainable
    for layer in model.layers:
        layer.trainable = True

    return model


# Initialize the model
print("Loading original Inception model...")
model = build_inception_model(input_shape, num_classes=2)
print("New model built successfully.")

# Uncomment to load pre-trained weights
# weights_name = 'model_inceptionv3_cyto_v16_5_eberhard_v3.15.h5'
# model.load_weights(weights_name)
# print(f"Weights loaded: {weights_name}")

# Define optimizer and compile the model
print("Compiling the model...")
optimizer = SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    optimizer=optimizer,
    loss="binary_crossentropy",
    metrics=[
        "binary_accuracy",
        AUC(),
        Precision(),
        Recall(),
        TruePositives(),
        TrueNegatives(),
        FalsePositives(),
        FalseNegatives(),
    ]
)
print("Model compiled successfully.")

In [ ]:
# Save model weights
weights_path = f"{model_name}.h5"
model.save_weights(weights_path)
print(f"Model weights saved to {weights_path}")

# Serialize model architecture to JSON
json_path = f"{model_name}.json"
model_json = model.to_json()
with open(json_path, "w") as json_file:
    json_file.write(model_json)
print(f"Model architecture saved to {json_path}")

In [ ]:
# Model checkpoint callback to save model weights after every epoch
checkpoint_filepath = f"{model_name}_epoch.{{epoch:02d}}.h5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    save_freq='epoch',
    save_best_only=False
)
print(f"Model checkpoints will be saved to: {checkpoint_filepath}")

# CSV Logger callback to log training history
csv_logger_filepath = f"{model_name}_model_history_log.csv"
csv_logger = CSVLogger(csv_logger_filepath, append=True)
print(f"Training history will be logged to: {csv_logger_filepath}")

# List of callbacks
callbacks_list = [model_checkpoint_callback, csv_logger]

##################
### Training #####
##################

print("Start training")
history = model.fit(
    x=train_generator,
    epochs=nr_epochs,
    validation_data=validation_generator,
    steps_per_epoch=nr_train_samples // batch_size,
    validation_steps=nr_val_samples // batch_size,
    callbacks=callbacks_list,
    workers=8,
    max_queue_size=16,
    use_multiprocessing=True
)

##############################
### Save Model and Weights ###
##############################

# Save model weights
weights_filepath = f"{model_name}.h5"
model.save_weights(weights_filepath)
print(f"Model weights saved to: {weights_filepath}")

# Serialize model architecture to JSON
model_json_filepath = f"{model_name}.json"
model_json = model.to_json()
with open(model_json_filepath, "w") as json_file:
    json_file.write(model_json)
print(f"Model architecture saved to: {model_json_filepath}")